# IML and xAI with SHAP

In this tutorial we will try to obtain explanations for the black-box/non-interpretable models using **[shap](https://shap.readthedocs.io/en/latest/) (SHapley Additive exPlanations)**. We will cover the basics of shap usage with the examples on tabular, image and text data.

## Tabular data

For the tabular data example we will apply [XGBoost](https://xgboost.readthedocs.io/en/stable/python/index.html) on [California Housing Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) to obtain the predictions of house prices. The explanations on how the prices of houses in particulare blocks are obtained are provided with the help of SHAP.

In [ ]:
# install shap
!pip install shap

In [ ]:
# import dependencies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

import shap

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
np.random.seed(42)

In [ ]:
# obtain the dataset for examination
# the dataframe is a standard one provided by scikit-learn
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
# print out the dataset description using the class descriptor
print(california_housing.DESCR)

In [ ]:
# examine the dataset
california_housing.frame.head()

In [ ]:
# look into the histograms of features
california_housing.frame.hist(figsize=(12, 10), bins=30, edgecolor="black")
plt.subplots_adjust(hspace=0.7, wspace=0.4)

For the sake of the tutorial we will not perform any manipulations with data, although there should be some correlated variables, as the purpose is to look into explainability of the model.

In [ ]:
# load the separated features and targets
X_tab, y_tab = fetch_california_housing(return_X_y=True, as_frame=True)

It is not necessary to perform the train_test_split to obtain shaply values. We just need a dataset and a model, that has been fitted to this dataset.

In [ ]:
# create XGBoost instance, parameters have been taken arbitraly to provide rather good performance
xgb = xgb.XGBRegressor(colsample_bytree = 1, eta=0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 2000)

# fit model to data
xgb.fit(X_tab, y_tab)

# make predictions on the same set to assess train performance
predictions = xgb.predict(X_tab)

print(f'Train MSE: {mean_squared_error(y_tab, predictions):.4f}')
print(f'Train R-Squared: {r2_score(y_tab, predictions):.4f}')

Here comes the most interesting part: generation of shapley values to explain the predictions. `shap` library contains many types of [explainers](https://shap.readthedocs.io/en/latest/api.html#explainers) for different kind of models, but we will focus on the following three:
- [Explainer](https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html#shap.Explainer) (a universal method to explain any ML model)
- [TreeExplainer](https://shap.readthedocs.io/en/latest/generated/shap.TreeExplainer.html) (to explain the output of ensemble tree models)
- [DeepExplainer](https://shap.readthedocs.io/en/latest/generated/shap.DeepExplainer.html) (to approximate SHAP values for deep learning models)


Predictibly, we will use TreeExplainer to obtain explanations for the XGBoost instance we've created above.

In [ ]:
# initialize the explainer with the model instance being the parameter
explainer_tab = shap.TreeExplainer(xgb)

# calculate approximate shapley values using our dataset of features
shap_values_tab = explainer_tab(X_tab)

How can we explain the shapley value: **Given the current set of feature values**, the contribution of a feature value to the difference between the actual prediction and the mean prediction is the estimated Shapley value.

The Shapley value is **NOT** the difference in prediction when we would remove the feature from the model.

The Shapley value returns a simple value per feature, but no prediction model like. It cannot be used to make statements about changes in prediction for changes in the input.


In [ ]:
# plot the explanation for a particular observation
shap.plots.waterfall(shap_values_tab[0])

In [ ]:
# initialize the necessary javascript libraries for interactive plots (in Google Colab should be used in every cell)
shap.plots.initjs()

# visualize the first prediction's explanation with a force plot
shap.plots.force(shap_values_tab[0])

In [ ]:
# initialize the necessary javascript libraries for interactive plots
shap.plots.initjs()

# visualize force plot for many observations
shap.plots.force(shap_values_tab[:50])

Average shapley values could be used as a global method to understand which features have the highest impact on the target variable. Note: the mean of *absolute* values is plotted.

In [ ]:
# average shap values
shap.plots.bar(shap_values_tab)

## Images

For the image explanation example we will use the same model as in Tutorial 2 with a small change: for ReLU layers the `inplace=False` is sey so that outputs of the ReLU were not overwriting the output of the previous action.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [ ]:
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()

        self.convolutional_layer = nn.Sequential(

            # convolutional layer 1
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, padding=0),
            nn.BatchNorm2d(10),
            nn.ReLU(inplace=False), # here is the change

            # max pooling layer 1
            nn.MaxPool2d(kernel_size=2, stride=2),

            # convolutional layer 2
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, padding=0),
            nn.BatchNorm2d(20),
            nn.ReLU(inplace=False), # here is the change

            # max pooling layer 2
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layer = nn.Sequential(
            nn.Linear(in_features=20*4*4, out_features=80),
            nn.ReLU(inplace=False), # here is the change
            nn.Linear(in_features=80, out_features=10),
        )

    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
          verbose: True if you want to print the shapes of the intermediate variables.

        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        x = self.convolutional_layer(x)
        x = x.view(-1, 20*4*4)
        x = self.linear_layer(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

# download the dataset
testset = torchvision.datasets.FashionMNIST(root='', train=False, download=True, transform=transform)

# define the labels of the images
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# specify the batch size for training and test
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [ ]:
# download the pre-saved checkpoint, the file name will be 'fashion_net.pth'
!gdown 1V1QQ2qSnvmmsn4cGI_orzCepmOeyUO-5

In [ ]:
# initialize our network
model_image = FashionNet()

# load trained models
model_image.load_state_dict(torch.load('fashion_net.pth', map_location=torch.device('cpu')))

In [ ]:
# create a loader with a batch of images
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)
images, true_labels = next(iter(test_loader))

# train images will be used to calculate the approximate shapley values
train_images = images[:120]

# images to assess shapley values
# make sure that the sum of train and test images are less than the batch_size
test_images = images[120:125]

# initialize the DeepExplainer with the model instance and images for training being the parameters
explainer_img = shap.DeepExplainer(model_image, train_images)

# calculate approximate shapley values using our test dataset
shap_values_img = explainer_img.shap_values(test_images)

In [ ]:
# create array for vizualizations
shap_img_numpy = [np.swapaxes(np.swapaxes(s, 1, -1), 1, 2) for s in shap_values_img]
test_img_numpy = np.swapaxes(np.swapaxes(test_images.numpy(), 1, -1), 1, 2)

# create labels for vizualizations
tl = [classes[idx] for idx in true_labels[120:125].numpy()]
l = [classes[:] for _ in range(5)]

# vizualize
shap.image_plot(shap_img_numpy, -test_img_numpy, labels=np.array(l), true_labels=tl)

We can see how each pixel contributes to the determination wheather the picture belongs to a particcular class. Using the same library it is possible to obtain explanations for particular layers of NN and partition the picture in some other manner.

## Text

For the text data we will see how particular words contribute to the classification of the text sequence. We will use a pre-trained transformer classification model from [hugginface](https://huggingface.co/nateraw/bert-base-uncased-emotion) and the [emotions dataset](https://huggingface.co/datasets/SetFit/emotion/)

In [ ]:
!pip install datasets transformers

In [ ]:
# import the dependencies
import datasets
import scipy as sp
import transformers

In [ ]:
# load the emotion dataset
dataset_txt = datasets.load_dataset("emotion", split="train")
data_txt = pd.DataFrame({"text": dataset_txt["text"], "emotion": dataset_txt["label"]})

In [ ]:
# load the model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
model_txt = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion")

# extract labels
labels = sorted(model_txt.config.label2id, key=model_txt.config.label2id.get)

# send model to cpu
model_txt.to('cpu')

print(f'Emotions fror classification: {labels}')

In [ ]:
# this defines a that takes a list of strings and outputs scores for each class
def f(texts):
    # create a tensor from the list of sequences and send it to cpu
    tv = torch.tensor(
        [
            tokenizer.encode(text, padding="max_length", max_length=128, truncation=True)
            for text in texts
        ]
    ).cpu()
    # create attention mask and send it to cpu
    attention_mask = (tv != 0).type(torch.int64).cpu()

    # calculate model output
    outputs = model_txt(tv, attention_mask=attention_mask)[0].detach().cpu().numpy()

    # calculate the scores for each class
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T

    # calculate the probabilities that the sequences belongs to the class
    val = sp.special.logit(scores)

    return val

In [ ]:
method = "transformers tokenizer"

# build an explainer by passing a transformers tokenizer
if method == "transformers tokenizer":
    explainer_txt = shap.Explainer(f, tokenizer, output_names=labels)

# build an explainer by explicitly creating a masker
elif method == "default masker":
    masker = shap.maskers.Text(r"\W")  # this will create a basic whitespace tokenizer
    explainer_txt = shap.Explainer(f, masker, output_names=labels)

In [ ]:
# calculate shapley values for the test data
shap_values_txt = explainer_txt(data_txt["text"][:3])

In [ ]:
# visualize
shap.plots.text(shap_values_txt)

Using shap it is possible to see how each particular word contributes to the identification of class